## A set of codes from camera parameter estimation to param.h5 generation

### 0. import libraries

In [ ]:
import numpy as np
import cv2
import glob
import os
import h5py
import os
from tqdm import tqdm

### 1. Camera parameter estimation

First, print the chessbord pattern ([chessboard_23mm.pdf](chessboard_23mm.pdf)) at actual size (no scaling). Record a video while moving the chessboard in front of cameras and save the video. Below the video file name is assumed to be "calib_vid.mp4". Then run the following code: 

In [ ]:
# utility functions

def analyze_chessboardvid(vid_path, square_size=23, saveimg=False, frame_intv=5):
    if saveimg:
        os.makedirs('./tmp/', exist_ok=True)

    # termination criteria
    criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
    # prepare object points, like (0,0,0), (1,0,0), (2,0,0) ....,(6,5,0)
    objp = np.zeros((6 * 9, 3), np.float32)
    objp[:, :2] = np.mgrid[0:9, 0:6].T.reshape(-1, 2) * square_size

    vf = vid_path

    objpoints = []  # 3d point in real world space
    imgpoints = []  # 2d points in image plane.
    cap = cv2.VideoCapture(vf)
    frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    im_w = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    im_h = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    for i_frame in tqdm(range(10, frame_count, frame_intv)):

        cap.set(cv2.CAP_PROP_POS_FRAMES, i_frame)

        ret, frame = cap.read()

        # Find the chess board corners
        gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        ret, corners = cv2.findChessboardCorners(gray, (9, 6))
        if ret == True:

            corners2 = cv2.cornerSubPix(gray, corners, (11, 11), (-1, -1), criteria)
            imgpoints.append(corners2)
            objpoints.append(objp)

            if saveimg:
                frame2 = cv2.drawChessboardCorners(frame, (9, 6), corners2, ret)
                outputfname = "./tmp/" + str(i_frame) + ".jpg"
                cv2.imwrite(outputfname, frame2)

    with h5py.File(os.path.splitext(vid_path)[0] + '_chessb_detection.h5', mode='w') as h5file:

        h5file.create_dataset('/imp', data=imgpoints)
        h5file.create_dataset('/objp', data=objpoints)
        h5file.create_dataset('/imsize', data=[im_w, im_h])

def calibrate_camera_intrinsic(vid_path, mtx_init=None, dist_init=None, outpath=None):

    if outpath is None:
        outpath = os.path.splitext(vid_path)[0] + '_cam_intrinsic.h5'

    with h5py.File(outpath, mode='w') as h5file_out:
        with h5py.File(os.path.splitext(vid_path)[0] + '_chessb_detection.h5', mode='r') as h5file:
            imgpoints = h5file['/imp'][()]
            objpoints = h5file['/objp'][()]
            imsize = h5file['/imsize'][()]

        # normal camera
        ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, (imsize[0], imsize[1]), mtx_init,
                                                           dist_init)
        dist = dist.ravel()

        h5file_out.create_dataset('/mtx', data=mtx)
        h5file_out.create_dataset('/dist', data=dist)
        h5file_out.create_dataset('/im_w', data=imsize[0])
        h5file_out.create_dataset('/im_h', data=imsize[1])

        print('[Result]')
        print('Camera matrix:')
        print(mtx)
        print('Distortion coeff:')
        print(dist)
        print('image size:')
        print(imsize)
        
        """
        # omnidir camera
        imgpoints2 = []
        objpoints2 = []
        for i in range(imgpoints.shape[0]):
            imgpoints2.append(imgpoints[i, :, :, :])
            objpoints2.append(np.reshape(objpoints[i, :, :], [-1, 1, 3]))

        calibration_flags = cv2.omnidir.CALIB_USE_GUESS + cv2.omnidir.CALIB_FIX_SKEW + cv2.omnidir.CALIB_FIX_CENTER

        rms, K, xi, D, rvecs, tvecs, idx = \
            cv2.omnidir.calibrate(
                objpoints2,
                imgpoints2,
                (imsize[0], imsize[1]),
                K=None,
                xi=None,
                D=None,
                flags=calibration_flags,
                criteria=(cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 200, 1e-8)
            )

        h5file_out.create_dataset('/K', data=K)
        h5file_out.create_dataset('/xi', data=xi)
        h5file_out.create_dataset('/D', data=D)
        """


In [ ]:
vid_path = 'calib_vid.mp4'
analyze_chessboardvid(vid_path, saveimg=True, frame_intv=5)

In [ ]:
calibrate_camera_intrinsic(vid_path, outpath='cam_param.h5')

For more info about camera calibration, see: https://docs.opencv.org/4.x/dc/dbb/tutorial_py_calibration.html

### 2. Making param.h5 file based on user inputs and the camera parameter.

In [ ]:
# utility functions

def create_paramfile(camera_calibration_file, paramfile_out_path, daq_fs=384000, daq_n_ch=4, camera_height=0.5, 
                     mic0pos=[0, 0, 0], speedOfSound = 343.0, pressure_calib=None):

    with h5py.File(camera_calibration_file, mode='r') as f:
        camera_matrix = f['/mtx'][()]
        dist_coeff = f['/dist'][()]
        im_w = f['/im_w'][()]
        im_h = f['/im_h'][()]

    fx, fy, ppx, ppy = camera_matrix[0][0], camera_matrix[1][1], camera_matrix[0][2], camera_matrix[1][2]

    if pressure_calib is None:
        pressure_calib = np.ones(daq_n_ch, dtype=float)

    mic0pos = np.array(mic0pos, dtype=float)
    r = np.array([1, 0, 0, 0, 1, 0, 0, 0, 1], dtype=float)
    t = np.array([0, 0, 0], dtype=float)

    with h5py.File(paramfile_out_path, mode='w') as f:
        f.create_dataset('/camera_param/camera_height', data=camera_height)
        f.create_dataset('/daq_param/fs', data=daq_fs)
        f.create_dataset('/daq_param/n_ch', data=daq_n_ch)
        f.create_dataset('/misc/speedOfSound', data=speedOfSound)
        f.create_dataset('/misc/pressure_calib', data=pressure_calib)
        f.create_dataset('/misc/mic0pos', data=mic0pos)
        f.create_dataset('/camera_param/color_intrin/coeffs', data=dist_coeff)
        f.create_dataset('/camera_param/color_intrin/fx', data=fx)
        f.create_dataset('/camera_param/color_intrin/fy', data=fy)
        f.create_dataset('/camera_param/color_intrin/width', data=im_w)
        f.create_dataset('/camera_param/color_intrin/height', data=im_h)
        f.create_dataset('/camera_param/color_intrin/ppx', data=ppx)
        f.create_dataset('/camera_param/color_intrin/ppy', data=ppy)
        f.create_dataset('/camera_param/depth_to_color_extrin/rotation', data=r)
        f.create_dataset('/camera_param/depth_to_color_extrin/translation', data=t)


In [ ]:

camera_calibration_file = 'cam_param.h5'
paramfile_out_path = 'param.h5'

### INPUT the other device info here ###########
daq_fs = 384000              # sampling rate in Hz
daq_n_ch = 4                 # number of channels
camera_height = 0.5          # in meters
mic0pos = [0.05, -0.05, 0.0]   # xyz position of mic ch-0 relative to the camera center in meters
speedOfSound = 343.0         # in meters / sec
################################################

create_paramfile(camera_calibration_file, paramfile_out_path, daq_fs, daq_n_ch, camera_height, mic0pos, speedOfSound)